# Transformers
Transformers first appeared around 2017 they are an attention based mdoel. The idea is that not all regions of the data are important and thus the model can pay attention to certain spaces within the dataset. 

- The first step is to compute the relvancy scores for between each vector. 
- The second step is to compute the sum of all vectors weighted by relevancy score.
- Then repeat for each vector in the sequence. 

This can also be thought about in a query key value model. This is used by search engines to find relevant documents. 

The output of the seatch is outputs = sum(inputs * pairwise_score(inputs, inputs)) this can be re written with value, query , key instead of the inputs. In   practice all the terms are the same. 


## Multihead attention
An expansion on attention where we conactenate three ir nire seperate attention heads tyo form a dense layer.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim = embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim)])
        self.layernorm1 = layers.LayerNormalization()
        self.layernorm2 = layers.LayerNormalization()
        
    def call(self,inputs,mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask = mask)
        proj_input = self.layernorm1(inputs+attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim":self.embed_dim, "dense_dim": self.dense_dim, "num_heads":self.num_heads})
        return config

2023-07-07 19:17:20.971176: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 19:17:21.756366: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
vocab_size = 20000
num_heads = 2
embed_dim = 256
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype = "int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

2023-07-07 19:17:22.422969: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 19:17:22.444386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 19:17:22.444571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 transformer_encoder (Trans  (None, None, 256)         543776    
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257   

In [3]:
max_length = 600
max_tokens = 20000
text_vec = layers.TextVectorization(max_tokens=max_tokens, output_mode = "int", output_sequence_length=max_length)
train_ds = keras.utils.text_dataset_from_directory("/home/kaspar/Documents/code/data/aclImdb/train", batch_size=32)
val_ds = keras.utils.text_dataset_from_directory("/home/kaspar/Documents/code/data/aclImdb/val",batch_size=32)
test_ds = keras.utils.text_dataset_from_directory("/home/kaspar/Documents/code/data/aclImdb/test",batch_size=32)
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vec.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)





Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
callbacks = keras.callbacks.ModelCheckpoint("transformer_encoder.keras")
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=1, callbacks=callbacks)
model = keras.models.load_model("transformer_encoder.keras", custom_objects = {"TransformerEncoder":TransformerEncoder})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 12s 15ms/step - loss: 0.3158 - accuracy: 0.8619
Test acc: 0.862


Note just giving it one session as there is little point training this thing yet since it still has no infomration about sequential information and thus will only ever get 87.5% accuracy.

In [9]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim = input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim = output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
        
    def call(self,inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta = 1)
        embed_tokens = self.token_embeddings(inputs)
        embed_positions = self.position_embeddings(positions)
        return embed_tokens + embed_positions
    
    def compute_mask(self, inputs, mask = None):
        return tf.math.not_equal(inputs,0)
    
    def get_config(self):
        config = super().get_config()
        config.update({"input_dim":self.input_dim, "sequence_length": self.sequence_length, "output_dim":self.output_dim})
        return config
    

In [10]:
vocab_size = 20000
num_heads = 2
embed_dim = 256
dense_dim = 32
sequence_length = 600

inputs = keras.Input(shape=(None,), dtype = "int64")
x = PositionalEmbedding(sequence_length,vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

callbacks = keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras")
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=5, callbacks=callbacks)
model = keras.models.load_model("full_transformer_encoder.keras", custom_objects = {"TransformerEncoder":TransformerEncoder, "PositionalEmbedding":PositionalEmbedding})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posi  (None, None, 256)         5273600   
 tionalEmbedding)                                                
                                                                 
 transformer_encoder_1 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_1 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

# Notes
So this transformer is now the full model. It can operates a reasonably good model. However note that it doesnt perform as good as bag of words in this instance. That is because the sequence len gths are relatively short for which bag of words is adventageous. 